The following tutorial is available on the [Wallaroo Github Repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2024.4_tutorials/wallaroo-run-anywhere/pipeline-edge-publish/edge-classification-cybersecurity).

## Classification Cybersecurity Services Edge Deployment Demonstration

This notebook will walk through building Wallaroo pipeline with a a Classification model deployed to detect the likelihood of a URL to be fraudulent, then publishing that pipeline to an Open Container Initiative (OCI) Registry where it can be deployed in other Docker and Kubernetes environments.

This demonstration will focus on deployment to the edge.  For further examples of using Wallaroo with this computer vision models, see [Wallaroo 101](https://docs.wallaroo.ai/wallaroo-101/).

This demonstration performs the following:

1. As a Data Scientist in Wallaroo Ops:
    1. Upload a computer vision model to Wallaroo, deploy it in a Wallaroo pipeline, then perform a sample inference.
    1. Publish the pipeline to an Open Container Initiative (OCI) Registry service.  This is configured in the Wallaroo instance.  See [Edge Deployment Registry Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/) for details on adding an OCI Registry Service to Wallaroo as the Edge Deployment Registry.  This demonstration uses a GitHub repository - see [Introduction to GitHub Packages](https://docs.github.com/en/packages/learn-github-packages/introduction-to-github-packages) for setting up your own package repository using GitHub, which can then be used with this tutorial.
    1. View the pipeline publish details.
1. As a DevOps Engineer in a remote aka edge device:
    1. Deploy the published pipeline as a Wallaroo Inference Server.  This example will use Docker.
    1. Perform a sample inference through the Wallaroo Inference Server with the same data used in the data scientist example.

## Data Scientist Pipeline Publish Steps

### Load Libraries

The first step is to import the libraries used in this notebook.

In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError

import pyarrow as pa
import pandas as pd
import requests


# used to display dataframe information without truncating
from IPython.display import display
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

### Connect to the Wallaroo Instance through the User Interface

The next step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [2]:
wl = wallaroo.Client()

### Create a New Workspace

We'll use the SDK below to create our workspace , assign as our **current workspace**, then display all of the workspaces we have at the moment.  We'll also set up variables for our models and pipelines down the road, so we have one spot to change names to whatever fits your organization's standards best.

To allow this tutorial to be run by multiple users in the same Wallaroo instance, a random 4 character prefix will be added to the workspace, pipeline, and model.  Feel free to set `suffix=''` if this is not required.

In [3]:
workspace_name = f'edge-publish-classification-cybersecurity'
pipeline_name = 'edge-pipeline-classification-cybersecurity'
model_name = 'aloha'
model_file_name = './models/alohacnnlstm.zip'

In [4]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)

wl.set_current_workspace(workspace)

{'name': 'edge-publish-classification-cybersecurity', 'id': 7, 'archived': False, 'created_by': '76b893ff-5c30-4f01-bd9e-9579a20fc4ea', 'created_at': '2024-04-22T17:04:19.54487+00:00', 'models': [], 'pipelines': []}

### Upload the Model

When a model is uploaded to a Wallaroo cluster, it is optimized and packaged to make it ready to run as part of a pipeline. In many times, the Wallaroo Server can natively run a model without any Python overhead. In other cases, such as a Python script, a custom Python environment will be automatically generated. This is comparable to the process of "containerizing" a model by adding a small HTTP server and other wrapping around it.

Our pretrained model is in ONNX format, which is specified in the `framework` parameter.

In [5]:
edge_demo_model = wl.upload_model(
    model_name,
    model_file_name,
    framework=wallaroo.framework.Framework.TENSORFLOW,
)

### Reserve Pipeline Resources

Before deploying an inference engine we need to tell wallaroo what resources it will need.
To do this we will use the wallaroo DeploymentConfigBuilder() and fill in the options listed below to determine what the properties of our inference engine will be.

We will be testing this deployment for an edge scenario, so the resource specifications are kept small -- what's the minimum needed to meet the expected load on the planned hardware.

- cpus - 4 => allow the engine to use 4 CPU cores when running the neural net
- memory - 4Gi => each inference engine will have 4 GB of memory, which is plenty for processing this much data.


In [6]:
deploy_config = (wallaroo
                 .DeploymentConfigBuilder()
                 .replica_count(1)
                 .cpus(4)
                 .memory("4Gi")
                 .build()
                 )

### Simulated Edge Deployment

We will now deploy our pipeline into the current Kubernetes environment using the specified resource constraints. This is a "simulated edge" deploy in that we try to mimic the edge hardware as closely as possible.

In [7]:
pipeline = wl.build_pipeline(pipeline_name)
display(pipeline)

# clear the steps if used before
pipeline.clear()
pipeline.add_model_step(edge_demo_model)

pipeline.deploy(deployment_config = deploy_config)

name,edge-pipeline-classification-cybersecurity
created,2024-04-22 17:04:23.058677+00:00
last_updated,2024-04-22 17:04:23.058677+00:00
deployed,(none)
arch,None
accel,None
tags,
versions,3f585deb-71bd-4a0a-b08b-4970d4e60a77
steps,
published,False


name,edge-pipeline-classification-cybersecurity
created,2024-04-22 17:04:23.058677+00:00
last_updated,2024-04-22 17:04:23.952878+00:00
deployed,True
arch,x86
accel,none
tags,
versions,"93ca84ca-9121-4933-b44c-95539fe5c0ee, 3f585deb-71bd-4a0a-b08b-4970d4e60a77"
steps,aloha
published,False


### Run Single Image Inference

A single image, encoded using the Apache Arrow format, is sent to the deployed pipeline. Arrow is used here because, as a binary protocol, there is far lower network and compute overhead than using JSON. The Wallaroo Server engine accepts both JSON, pandas DataFrame, and Apache Arrow formats.

The sample DataFrames and arrow tables are in the `./data` directory.  We'll use the Apache Arrow table `data_25k.arrow`.

Once complete, we'll display how long the inference request took.

In [8]:
deploy_url = pipeline._deployment._url()

headers = wl.auth.auth_header()

headers['Content-Type']='application/vnd.apache.arrow.file'
# headers['Content-Type']='application/json; format=pandas-records'
headers['Accept']='application/json; format=pandas-records'

dataFile = './data/data_1k.arrow'

In [9]:
!curl -X POST {deploy_url} \
     -H "Authorization:{headers['Authorization']}" \
     -H "Content-Type:{headers['Content-Type']}" \
     -H "Accept:{headers['Accept']}" \
     --data-binary @{dataFile} > curl_response.df.json

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1088k  100  892k  100  196k   740k   163k  0:00:01  0:00:01 --:--:--  904k    0   207k --:--:-- --:--:-- --:--:--  207k


### Undeploy Pipeline

We will now undeploy the pipeline with the testing complete.

In [10]:
pipeline.undeploy()

name,edge-pipeline-classification-cybersecurity
created,2024-04-22 17:04:23.058677+00:00
last_updated,2024-04-22 17:04:23.952878+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"93ca84ca-9121-4933-b44c-95539fe5c0ee, 3f585deb-71bd-4a0a-b08b-4970d4e60a77"
steps,aloha
published,False


### Publish the Pipeline for Edge Deployment

It worked! For a demo, we'll take working once as "tested". So now that we've tested our pipeline, we are ready to publish it for edge deployment.

Publishing it means assembling all of the configuration files and model assets and pushing them to an Open Container Initiative (OCI) repository set in the Wallaroo instance as the Edge Registry service.  DevOps engineers then retrieve that image and deploy it through Docker, Kubernetes, or similar deployments.

See [Edge Deployment Registry Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/) for details on adding an OCI Registry Service to Wallaroo as the Edge Deployment Registry.

This is done through the SDK command `wallaroo.pipeline.publish(deployment_config)` which has the following parameters and returns.

#### Publish a Pipeline Parameters

The `publish` method takes the following parameters.  The containerized pipeline will be pushed to the Edge registry service with the model, pipeline configurations, and other artifacts needed to deploy the pipeline.

| Parameter | Type | Description |
|---|---|---|
| `deployment_config` | `wallaroo.deployment_config.DeploymentConfig` (*Optional*) | Sets the pipeline deployment configuration.  For example:    For more information on pipeline deployment configuration, see the [Wallaroo SDK Essentials Guide: Pipeline Deployment Configuration](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-deployment-config/).

#### Publish a Pipeline Returns

| Field | Type | Description |
|---|---|---|
| id | integer | Numerical Wallaroo id of the published pipeline. |
| pipeline version id | integer | Numerical Wallaroo id of the pipeline version published. |
| status | string | The status of the pipeline publication.  Values include:  <ul><li>PendingPublish: The pipeline publication is about to be uploaded or is in the process of being uploaded.</li><li>Published:  The pipeline is published and ready for use.</li></ul> |
| Engine URL | string | The URL of the published pipeline engine in the edge registry. |
| Pipeline URL | string | The URL of the published pipeline in the edge registry. |
| Helm Chart URL | string | The URL of the helm chart for the published pipeline in the edge registry. |
| Helm Chart Reference | string | The help chart reference. |
| Helm Chart Version | string | The version of the Helm Chart of the published pipeline.  This is also used as the Docker tag. |
| Engine Config | `wallaroo.deployment_config.DeploymentConfig` | The pipeline configuration included with the published pipeline. |
| Replaces | List | A list of any pipeline publishes replaced by this new publish. |
| Docker Run | string | Template of commands for deploying via `docker run`. |
| Helm Install | string | Template of commands for deploying via `helm`. |
| Created At | DateTime | When the published pipeline was created. |
| Updated At | DateTime | When the published pipeline was updated. |

### Publish Example

We will now publish the pipeline to our Edge Deployment Registry with the `pipeline.publish(deployment_config)` command.  `deployment_config` is an optional field that specifies the pipeline deployment.  This can be overridden by the DevOps engineer during deployment.

In [11]:
pub=pipeline.publish(deploy_config)
pub

Waiting for pipeline publish... It may take up to 600 sec.
Pipeline is publishing....... Published.


ID,5
Pipeline Name,edge-pipeline-classification-cybersecurity
Pipeline Version,21d4aa33-3a9a-4e55-9feb-6689eb822b7a
Status,Published
Engine URL,ghcr.io/wallaroolabs/doc-samples/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.1.0-main-4963
Pipeline URL,ghcr.io/wallaroolabs/doc-samples/pipelines/edge-pipeline-classification-cybersecurity:21d4aa33-3a9a-4e55-9feb-6689eb822b7a
Helm Chart URL,oci://ghcr.io/wallaroolabs/doc-samples/charts/edge-pipeline-classification-cybersecurity
Helm Chart Reference,ghcr.io/wallaroolabs/doc-samples/charts@sha256:981a67aee2754409b3c41ad3763d7910695008d032519ba44b7c5ac798ed07c2
Helm Chart Version,0.0.1-21d4aa33-3a9a-4e55-9feb-6689eb822b7a
Engine Config,"{'engine': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 1.0, 'memory': '512Mi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}, 'engineAux': {'autoscale': {'type': 'none'}, 'images': {}}}"
User Images,[]


In [12]:
display(pipeline)

name,edge-pipeline-classification-cybersecurity
created,2024-04-22 17:04:23.058677+00:00
last_updated,2024-04-22 17:05:19.451533+00:00
deployed,False
arch,x86
accel,none
tags,
versions,"21d4aa33-3a9a-4e55-9feb-6689eb822b7a, 93ca84ca-9121-4933-b44c-95539fe5c0ee, 3f585deb-71bd-4a0a-b08b-4970d4e60a77"
steps,aloha
published,False


### List Published Pipeline

The method `wallaroo.client.list_pipelines()` shows a list of all pipelines in the Wallaroo instance, and includes the `published` field that indicates whether the pipeline was published to the registry (`True`), or has not yet been published (`False`).

In [13]:
wl.list_pipelines()

[{'name': 'edge-pipeline-classification-cybersecurity', 'create_time': datetime.datetime(2024, 4, 22, 17, 4, 23, 58677, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'vgg16-clustering-pipeline', 'create_time': datetime.datetime(2024, 4, 22, 16, 17, 8, 651909, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'new-edge-inline-replacement', 'create_time': datetime.datetime(2024, 4, 22, 15, 43, 4, 574641, tzinfo=tzutc()), 'definition': '[]'},
 {'name': 'edge-inline-replacement-demon', 'create_time': datetime.datetime(2024, 4, 22, 15, 27, 36, 828826, tzinfo=tzutc()), 'definition': '[]'}]

### List Publishes from a Pipeline

All publishes created from a pipeline are displayed with the `wallaroo.pipeline.publishes` method.  The `pipeline_version_id` is used to know what version of the pipeline was used in that specific publish.  This allows for pipelines to be updated over time, and newer versions to be sent and tracked to the Edge Deployment Registry service.

#### List Publishes Parameters

N/A

#### List Publishes Returns

A List of the following fields:

| Field | Type | Description |
|---|---|---|
| id | integer | Numerical Wallaroo id of the published pipeline. |
| pipeline_version_id | integer | Numerical Wallaroo id of the pipeline version published. |
| engine_url | string | The URL of the published pipeline engine in the edge registry. |
| pipeline_url | string | The URL of the published pipeline in the edge registry. |
| created_by | string | The email address of the user that published the pipeline.
| Created At | DateTime | When the published pipeline was created. |
| Updated At | DateTime | When the published pipeline was updated. |

In [14]:
pipeline.publishes()

id,pipeline_version_name,engine_url,pipeline_url,created_by,created_at,updated_at
5,21d4aa33-3a9a-4e55-9feb-6689eb822b7a,ghcr.io/wallaroolabs/doc-samples/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.1.0-main-4963,ghcr.io/wallaroolabs/doc-samples/pipelines/edge-pipeline-classification-cybersecurity:21d4aa33-3a9a-4e55-9feb-6689eb822b7a,john.hummel@wallaroo.ai,2024-22-Apr 17:05:20,2024-22-Apr 17:05:20


## DevOps Deployment

We now have our pipeline published to our Edge Registry service.  We can deploy this in a x86 environment running Docker that is logged into the same registry service that we deployed to.

For more details, check with the documentation on your artifact service.  The following are provided for the three major cloud services:

* [Set up authentication for Docker](https://cloud.google.com/artifact-registry/docs/docker/authentication)
* [Authenticate with an Azure container registry](https://learn.microsoft.com/en-us/azure/container-registry/container-registry-authentication?tabs=azure-cli)
* [Authenticating Amazon ECR Repositories for Docker CLI with Credential Helper](https://aws.amazon.com/blogs/compute/authenticating-amazon-ecr-repositories-for-docker-cli-with-credential-helper/)

Once a pipeline is deployed to the Edge Registry service, it can be deployed in environments such as Docker, Kubernetes, or similar container running services by a DevOps engineer.  For full details, see [How to Publish and Deploy AI Workloads for For Edge/Multicloud Model Deployments](https://docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-run-anywhere/wallaroo-model-operations-run-anywhere-deploy/).  The pipeline publishes `Docker Run Command` and `Helm Install Command` provide templates for deployment.

## Edge Deployed Pipeline API Endpoints

Once deployed, we can check the pipelines and models available.  We'll use a `curl` command, but any HTTP based request will work the same way.

The endpoint `/pipelines` returns:

* **id** (*String*):  The name of the pipeline.
* **status** (*String*):  The status as either `Running`, or `Error` if there are any issues.

For this example, the deployment is made on a machine called `testboy.local`.  Replace this URL with the URL of you edge deployment.


In [16]:
!curl testboy.local:8081/pipelines

{"pipelines":[{"id":"edge-pipeline-classification-cybersecurity","version":"21d4aa33-3a9a-4e55-9feb-6689eb822b7a","status":"Running"}]}

The endpoint `/models` returns a List of models with the following fields:

* **name** (*String*): The model name.
* **sha** (*String*): The sha hash value of the ML model.
* **status** (*String*):  The status of either Running or Error if there are any issues.
* **version** (*String*):  The model version.  This matches the version designation used by Wallaroo to track model versions in UUID format.

In [18]:
!curl testboy.local:8081/models

{"models":[{"sha":"d71d9ffc61aaac58c2b1ed70a2db13d1416fb9d3f5b891e5e4e2e97180fe22f8","name":"aloha","version":"9f853289-d667-480f-87e3-2d5532142963","status":"Running"}]}

### Edge Inference Endpoint

The inference endpoint takes the following pattern:

* `/infer`:  The inference endpoint remains the same regardless of the pipeline or models deployed.  This allows publish replacements without altering the inference endpoint used by other applications.

Wallaroo inference endpoint URLs accept the following data inputs through the `Content-Type` header:

* `Content-Type: application/vnd.apache.arrow.file`: For Apache Arrow tables.
* `Content-Type: application/json; format=pandas-records`: For pandas DataFrame in record format.

Once deployed, we can perform an inference through the deployment URL.

The endpoint returns `Content-Type: application/json; format=pandas-records` by default with the following fields:

* **check_failures** (*List[Integer]*): Whether any validation checks were triggered.  For more information, see [Wallaroo SDK Essentials Guide: Pipeline Management: Anomaly Testing](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/#anomaly-testing).
* **elapsed** (*List[Integer]*): A list of time in nanoseconds for:
  * [0] The time to serialize the input.
  * [1...n] How long each step took.
* **model_name** (*String*): The name of the model used.
* **model_version** (*String*): The version of the model in UUID format.
* **original_data**: The original input data.  Returns `null` if the input may be too long for a proper return.
* **outputs** (*List*): The outputs of the inference result separated by data type, where each data type includes:
  * **data**: The returned values.
  * **dim** (*List[Integer]*): The dimension shape returned.
  * **v** (*Integer*): The vector shape of the data.
* **pipeline_name**  (*String*): The name of the pipeline.
* **shadow_data**: Any shadow deployed data inferences in the same format as **outputs**.
* **time** (*Integer*): The time since UNIX epoch.

In [20]:
!curl -X POST testboy.local:8081/infer \
    -H "Content-Type: application/vnd.apache.arrow.file" \
        --data-binary @./data/data_1k.arrow

[{"check_failures":[],"elapsed":[373732,442446319],"model_name":"aloha","model_version":"9f853289-d667-480f-87e3-2d5532142963","original_data":null,"outputs":[{"Float":{"data":[0.001519581419415772,0.000028375030524330214,3.0770573289373715e-7,8.822828535468008e-13,5.48706066183513e-6,8.510186136383105e-19,8.317945598353995e-16,2.3569645083006874e-18,1.0514139558992497e-9,0.0002814377367030829,1.0622999724319016e-7,2.936564633495599e-14,8.164665775065028e-14,1.3123198083775378e-9,0.00003420593202463351,0.002541592111811042,1.1404551969462773e-8,4.512332076999498e-13,7.114560389709368e-7,9.998708099272138e-15,1.170912008853641e-16,1.382829804439864e-13,5.783209644505405e-7,6.076513958674923e-9,0.0005951693165116012,1.244429307068292e-16,2.1691126762731503e-16,6.757206924907957e-16,2.4466361381314528e-8,0.00011290510155959055,3.9228136294866545e-9,2.2086266192669624e-13,4.1538639390138743e-13,9.473686418681737e-11,0.0001534186740173027,1.1467229249495503e-10,4.7972046474109575e-11,2.0239